# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

data_path = '../data/Part02/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

train_num = train_Y.shape[0]

# 只留類別型欄位
df_objects = df[object_features]
df_objects = df_objects.fillna('None')

# 只留數值型欄位
df_numbers = df.drop(object_features, axis = 1)
df_numbers = df_numbers.fillna(0)[:train_num]
MMEncoder = MinMaxScaler()
df_numbers = pd.DataFrame(MMEncoder.fit_transform(df_numbers), columns = df_numbers.columns)

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



D:\Applications\Python\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
df_objects.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [4]:
df_numbers.head()

,Pclass,Age,SibSp,Parch,Fare
0,1.0,0.2750,0.125,0.0,0.014151
1,0.0,0.4750,0.125,0.0,0.139136
2,1.0,0.3250,0.000,0.0,0.015469
3,0.0,0.4375,0.125,0.0,0.103644
4,1.0,0.4375,0.000,0.0,0.015713


# 作業2
* 承上題，三者比較效果何者最好?

### Object featuer with 'Cabin'
* 均值 (0.8317) > 計數 (0.789) > 雜湊 (0.7766) > 標籤 (0.78)

### Number feature with 'Cabin'
* 均值 (0.7487) > 標籤 (0.7072) > 雜湊 (0.7005) > 計數 (0.6926)

In [5]:
#
df_data = pd.DataFrame()
df_data['Cabin'] = df_objects['Cabin'][:train_num]

## Label encoding
df_data['Cabin_Label'] = LabelEncoder().fit_transform(df_data['Cabin'])

### for other labels
for c in df_objects.columns:
    df_objects[c] = LabelEncoder().fit_transform(df_objects[c])
    
df_objects = df_objects[:train_num]


## Counting encoding
count_df = df_data.groupby(['Cabin'])['Cabin'].agg({'Cabin_Count':'size'}).reset_index()
df_data = pd.merge(df_data, count_df, on=['Cabin'], how='left')

## Hash encoding
df_data['Cabin_Hash'] = df_data['Cabin'].map(lambda x:hash(x) % 10)

## Mean encoding
data = pd.concat([df_data[:train_num], train_Y], axis=1)
mean_df = data.groupby(['Cabin'])['Survived'].mean().reset_index()
mean_df.columns = ['Cabin', 'Cabin_Mean']
df_data = pd.merge(df_data, mean_df, on='Cabin', how='left')

df_objects.head()

D:\Applications\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  app.launch_new_instance()


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


# Only object features with 'Cabin'

In [6]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
train_X = df_objects.copy()
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.780004837244799


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [7]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
train_X = df_objects.copy()
train_X['Cabin_Count'] = df_data['Cabin_Count']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.7889812599320445


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,155,1,720,185,3,687
1,286,0,816,106,0,1
2,523,0,914,185,3,687
3,422,0,65,70,3,2
4,22,1,649,185,3,687


In [8]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
train_X = df_objects.copy()
train_X['Cabin_Hash'] = df_data['Cabin_Hash']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.7766277027307382


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,5
1,286,0,816,106,0,7
2,523,0,914,185,3,5
3,422,0,65,70,3,2
4,22,1,649,185,3,5


In [9]:
# 'Cabin'均值編碼 + 邏輯斯迴歸
train_X = df_objects.copy()
train_X['Cabin_Mean'] = df_data['Cabin_Mean']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.8316784565642724


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Mean
0,155,1,720,185,3,0.299854
1,286,0,816,106,0,1.000000
2,523,0,914,185,3,0.299854
3,422,0,65,70,3,0.500000
4,22,1,649,185,3,0.299854


In [10]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
train_X = df_objects.copy()
train_X['Cabin_Count'] = df_data['Cabin_Count']
train_X['Cabin_Hash'] = df_data['Cabin_Hash']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.7811223684625824


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count,Cabin_Hash
0,155,1,720,185,3,687,5
1,286,0,816,106,0,1,7
2,523,0,914,185,3,687,5
3,422,0,65,70,3,2,2
4,22,1,649,185,3,687,5


In [11]:
# 'Cabin'計數編碼 + 'Cabin'均值編碼 + 邏輯斯迴歸
train_X = df_objects.copy()
train_X['Cabin_Count'] = df_data['Cabin_Count']
train_X['Cabin_Mean'] = df_data['Cabin_Mean']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.8305611381285182


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count,Cabin_Mean
0,155,1,720,185,3,687,0.299854
1,286,0,816,106,0,1,1.000000
2,523,0,914,185,3,687,0.299854
3,422,0,65,70,3,2,0.500000
4,22,1,649,185,3,687,0.299854


In [12]:
# 'Cabin'特徵雜湊 + 'Cabin'均值編碼 + 邏輯斯迴歸
train_X = df_objects.copy()
train_X['Cabin_Hash'] = df_data['Cabin_Hash']
train_X['Cabin_Mean'] = df_data['Cabin_Mean']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.8316784565642724


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash,Cabin_Mean
0,155,1,720,185,3,5,0.299854
1,286,0,816,106,0,7,1.000000
2,523,0,914,185,3,5,0.299854
3,422,0,65,70,3,2,0.500000
4,22,1,649,185,3,5,0.299854


# Only number featuers with Cabin

In [13]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
train_X = df_numbers.copy()
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.707234411677194


,Pclass,Age,SibSp,Parch,Fare
0,1.0,0.2750,0.125,0.0,0.014151
1,0.0,0.4750,0.125,0.0,0.139136
2,1.0,0.3250,0.000,0.0,0.015469
3,0.0,0.4375,0.125,0.0,0.103644
4,1.0,0.4375,0.000,0.0,0.015713


In [14]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
train_X = df_numbers.copy()
train_X['Cabin_Count'] = df_data['Cabin_Count']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.6926212511796104


,Pclass,Age,SibSp,Parch,Fare,Cabin_Count
0,1.0,0.2750,0.125,0.0,0.014151,687
1,0.0,0.4750,0.125,0.0,0.139136,1
2,1.0,0.3250,0.000,0.0,0.015469,687
3,0.0,0.4375,0.125,0.0,0.103644,2
4,1.0,0.4375,0.000,0.0,0.015713,687


In [15]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
train_X = df_numbers.copy()
train_X['Cabin_Hash'] = df_data['Cabin_Hash']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.7004864906462793


,Pclass,Age,SibSp,Parch,Fare,Cabin_Hash
0,1.0,0.2750,0.125,0.0,0.014151,5
1,0.0,0.4750,0.125,0.0,0.139136,7
2,1.0,0.3250,0.000,0.0,0.015469,5
3,0.0,0.4375,0.125,0.0,0.103644,2
4,1.0,0.4375,0.000,0.0,0.015713,5


In [16]:
# 'Cabin'均值編碼 + 邏輯斯迴歸
train_X = df_numbers.copy()
train_X['Cabin_Mean'] = df_data['Cabin_Mean']
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
train_X.head()

0.748682258624499


,Pclass,Age,SibSp,Parch,Fare,Cabin_Mean
0,1.0,0.2750,0.125,0.0,0.014151,0.299854
1,0.0,0.4750,0.125,0.0,0.139136,1.000000
2,1.0,0.3250,0.000,0.0,0.015469,0.299854
3,0.0,0.4375,0.125,0.0,0.103644,0.500000
4,1.0,0.4375,0.000,0.0,0.015713,0.299854
